In [1]:
!pip install beautifulsoup4
!pip install requests
!pip install yfinance
!pip install datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 8.4 MB/s eta 0:00:00


In [2]:
import yfinance as yf
import requests
import bs4 as bs
import datetime as dt
import pandas as pd

https://wire.insiderfinance.io/how-to-get-all-stocks-from-the-s-p500-in-python-fbe5f9cb2b61 to get list of s&p500 companies

In [3]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

tickers = []

for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

In [4]:
tickers[0]

'MMM\n'

As we can see the tickers all immported witha '\n'.

In [5]:
tickers = [s.replace('\n', '') for s in tickers]
tickers[0]

'MMM'

In [6]:
data = yf.download(tickers, start="2020-01-06", end="2023-06-30", interval = '1wk')
data.head()

[*********************100%***********************]  503 of 503 completed

ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1wk 2020-01-06 -> 2023-06-30)')
ERROR:yfinance:['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


Adj Close                                               \
                             A        AAL         AAP       AAPL       ABBV   
Date                                                                          
2020-01-06 00:00:00  85.549843  27.219410  138.306641  75.785126  75.242340   
2020-01-13 00:00:00  88.020920  28.295433  140.200150  77.836487  74.346786   
2020-01-20 00:00:00  86.204247  27.538231  131.503159  77.733902  71.532303   
2020-01-27 00:00:00  80.637009  26.741179  122.286415  75.584885  69.374527   
2020-02-03 00:00:00  81.232803  28.275507  123.381653  78.153961  79.024628   

                                                                              \
                           ABC        ABT       ACGL         ACN        ADBE   
Date                                                                           
2020-01-06 00:00:00  82.691467  80.235649  43.529999  196.864883  339.809998   
2020-01-13 00:00:00  86.510643  83.657135  45.169998  198.324799  349.739990   
2020-01-20 00:00:00  84.477547  85.333992  44.990002  198.491989  351.369995   
2020-01-27 00:00:00  81.285423  82.256683  44.160000  195.294312  351.140015   
2020-02-03 00:00:00  87.346672  82.945770  46.150002  201.356522  366.089996   

                     ...    Volume                                          \
                     ...      WYNN       XEL        XOM     XRAY       XYL   
Date                 ...                                                     
2020-01-06 00:00:00  ...   7640700  18697800   79414700  4594800   4721400   
2020-01-13 00:00:00  ...   9774400  11434400   69089600  6052900   6067700   
2020-01-20 00:00:00  ...  17169400  11546400   72860200  5690900   4509500   
2020-01-27 00:00:00  ...  18400100  14694000  107926100  6216000   5419600   
2020-02-03 00:00:00  ...  16803600  17431100  122578500  5683000  11993100   

                                                                     
                          YUM      ZBH     ZBRA      ZION       ZTS  
Date                                                                 
2020-01-06 00:00:00   7616700  5075325  2638300   8173300   8649700  
2020-01-13 00:00:00  10453400  3931716  2060000   9359700   9322000  
2020-01-20 00:00:00   9912500  3412081  1558300  13985300  10777700  
2020-01-27 00:00:00   8505200  4978196  1668800  10003300   9517100  
2020-02-03 00:00:00  10749300  8665390  1550900  12017000   9712000  

[5 rows x 3018 columns]

In [7]:
df = data.stack().reset_index().rename(index=str, columns={"level_1": "Symbol"}).sort_values(['Symbol','Date'])
df.reset_index()
df.head()

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
0,2020-01-06,A,85.549843,87.589996,88.239998,83.599998,84.000000,8855200.0
496,2020-01-13,A,88.020920,90.120003,90.279999,86.699997,87.809998,10250000.0
992,2020-01-20,A,86.204247,88.260002,90.639999,87.580002,89.800003,6756000.0
1488,2020-01-27,A,80.637009,82.559998,88.360001,82.339996,86.540001,10528800.0
1984,2020-02-03,A,81.232803,83.169998,85.500000,82.110001,83.290001,8224000.0


Flatten multi level index


In [8]:
df.to_csv('/tmp/weeklytarget2020_062023.csv')